In [5]:
%load_ext autoreload

In [32]:
%autoreload

from allennlp.commands.elmo import ElmoEmbedder
import os
from sys import path
path.append('../..')
from relation_extraction.data import utils
import h5py
import numpy as np
dataset = 'i2b2' #TODO: update this
data_path = '/crimea/geeticka/data/relation_extraction/' + dataset + '/'
def res(path): return os.path.join(data_path, path)
out_path = 'pre-processed/original/elmo/'

In [33]:
if not os.path.exists(res(out_path)):
    os.makedirs(res(out_path))
if not os.path.exists(res(out_path + 'input-sentences')):
    os.makedirs(res(out_path + 'input-sentences'))

Weights and options for Semeval

In [24]:
# OPTIONS_FILE = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
# WEIGHT_FILE = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"

Weights and options for the PubMed

In [25]:
OPTIONS_FILE = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pubmed/elmo_2x4096_512_2048cnn_2xhighway_options.json'
WEIGHT_FILE = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pubmed/elmo_2x4096_512_2048cnn_2xhighway_weights_PubMed_only.hdf5'

In [26]:
elmo = ElmoEmbedder(weight_file=WEIGHT_FILE, options_file=OPTIONS_FILE, cuda_device=3)

## Getting all of the ELMo layers

In [34]:
# border_sizes = [-1, 50, 1, 20]
border_sizes = [-1]

In [35]:
def get_sentences(filename, border_size):
    data = open(res('pre-processed/original/' + filename))
    data = utils.split_data_cut_sentence(data, border_size)
    sentences = data[0]
    return sentences

In [36]:
def write_sentences_to_txt(sentences, filename):
    with open(res(out_path + 'input-sentences/'+filename + '.txt'), 'w') as f:
        for sentence in sentences:
            sentence_string = " ".join(sentence)
            f.write(sentence_string + "\n")

In [37]:
def write_hdf5(filename):
    with open(res(out_path + 'input-sentences/' + filename + '.txt')) as file:
        elmo.embed_file(input_file=file, 
                        output_file_path = res(out_path + filename + '.hdf5'))

In [38]:
for border_size in border_sizes:
    #train_sentences = get_sentences('train_original.txt', border_size)
    #test_sentences = get_sentences('test_original.txt', border_size)
    #write_sentences_to_txt(train_sentences, 'train_original_border_' + str(border_size))
    #write_sentences_to_txt(test_sentences, 'test_original_border_' + str(border_size))
    write_hdf5('train_original_border_' + str(border_size))
    write_hdf5('test_original_border_' + str(border_size))

7833it [00:43, 181.16it/s]
14508it [01:21, 179.08it/s]
